In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
training_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
print(training_data)
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
print(test_data)

     PassengerId HomePlanet CryoSleep     Cabin    Destination   Age    VIP  \
0        0001_01     Europa     False     B/0/P    TRAPPIST-1e  39.0  False   
1        0002_01      Earth     False     F/0/S    TRAPPIST-1e  24.0  False   
2        0003_01     Europa     False     A/0/S    TRAPPIST-1e  58.0   True   
3        0003_02     Europa     False     A/0/S    TRAPPIST-1e  33.0  False   
4        0004_01      Earth     False     F/1/S    TRAPPIST-1e  16.0  False   
...          ...        ...       ...       ...            ...   ...    ...   
8688     9276_01     Europa     False    A/98/P    55 Cancri e  41.0   True   
8689     9278_01      Earth      True  G/1499/S  PSO J318.5-22  18.0  False   
8690     9279_01      Earth     False  G/1500/S    TRAPPIST-1e  26.0  False   
8691     9280_01     Europa     False   E/608/S    55 Cancri e  32.0  False   
8692     9280_02     Europa     False   E/608/S    TRAPPIST-1e  44.0  False   

      RoomService  FoodCourt  ShoppingMall     Spa 

In [3]:

Y_train = training_data.Transported
#print(Y_train)
X_train = training_data.drop(['Transported'], axis=1)

# Remove this line once we're done with testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train ,random_state = 42,test_size=0.2)

print(X_train)
# Transported does not exist in test data
#Y_test = test_data.Transported
#print(Y_test)
#X_test = test_data.drop(['Transported'], axis=1)
#X_test = test_data
#print(X_test)

     PassengerId HomePlanet CryoSleep     Cabin    Destination   Age    VIP  \
2333     2513_01      Earth     False       NaN    TRAPPIST-1e  28.0  False   
2589     2774_02      Earth     False   F/575/P    TRAPPIST-1e  17.0  False   
8302     8862_04     Europa      True   C/329/S    55 Cancri e  28.0  False   
8177     8736_02       Mars     False  F/1800/P    TRAPPIST-1e  20.0  False   
500      0539_02     Europa      True    C/18/P    55 Cancri e  36.0  False   
...          ...        ...       ...       ...            ...   ...    ...   
5734     6076_01      Earth       NaN   G/988/S    TRAPPIST-1e  18.0  False   
5191     5537_01       Mars     False  F/1063/S    TRAPPIST-1e  50.0    NaN   
5390     5756_06      Earth     False  F/1194/P  PSO J318.5-22  22.0  False   
860      0925_01       Mars     False   F/191/P    TRAPPIST-1e  34.0  False   
7270     7775_01     Europa     False   C/253/P    55 Cancri e  28.0  False   

      RoomService  FoodCourt  ShoppingMall    Spa  

In [4]:
#df[['A', 'B']] = df['AB'].str.split(' ', n=1, expand=True)
#print(type(X_train['Cabin'][0]))
X_train[['CabinDeck','CabinNum','CabinSide']] = X_train['Cabin'].str.split('/', n=-1, expand=True)
X_train.CabinNum = pd.to_numeric(X_train.CabinNum)
X_train.drop('Cabin',axis=1, inplace=True)
#print(X_train)
X_test[['CabinDeck','CabinNum','CabinSide']] = X_test['Cabin'].str.split('/', n=-1, expand=True)
X_test.CabinNum = pd.to_numeric(X_test.CabinNum)
X_test.drop('Cabin',axis=1, inplace=True)
#print(X_test)

In [5]:
unimportantColumns = ['PassengerId','Name']
X_train.drop(columns = unimportantColumns, inplace=True)
#print(X_train)
X_test.drop(columns = unimportantColumns, inplace=True)
#print('\nX_test')
#print(X_test)

In [6]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

encoder = ce.OneHotEncoder(handle_unknown='ignore', cols=object_cols, use_cat_names=True, return_df=True)
X_train =  encoder.fit_transform(X_train)
X_test =  encoder.transform(X_test)
#print(X_train)

Categorical variables:
['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'CabinDeck', 'CabinSide']


In [7]:
#OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
#OH_encoder.fit(X_train[object_cols])

#def my_OHenc_function(OH_encoder, train, object_cols):
#    OH_cols_train = pd.DataFrame(OH_encoder.transform(train[object_cols]))
#    OH_cols_train.index = train.index
#    OH_cols_train.columns = OH_encoder.get_feature_names_out()
#    num_train = train.drop(object_cols, axis=1)
#    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
#    #OH_train.columns = OH_train.columns.astype(str)
#    return OH_train

#X_train = my_OHenc_function(OH_encoder, X_train, object_cols)
#print(X_train)

#X_test = my_OHenc_function(OH_encoder, X_test, object_cols)
#print(X_test)

In [8]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_test))

X_train = imputed_X_train
X_test = imputed_X_valid

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestClassifier(max_depth=10, n_estimators=100, random_state=0)
forest_model.fit(X_train, Y_train)
predictions = forest_model.predict(X_test)
#print(predictions.shape)
#print(Y_test.shape)
#print(mean_absolute_error(Y_test, predictions))
predLen = len(predictions)
isGoodCount = 0
for x in range(predLen):
    isGood = Y_test.iloc[x] == predictions[x]
    if (isGood):
        isGoodCount+=1
    #print(x, Y_test.iloc[x], predictions[x], isGood)

print(isGoodCount,'/',predLen,'=', isGoodCount/predLen,'%')

1386 / 1739 = 0.79700977573318 %
